### Import packages

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import re
import requests
import os

#### Initialize headless browser

In [ ]:
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

#### Load the website 

In [ ]:
artist_name = 'annibalesiconolfi' # Replace with the desired artist name for example: annibalesiconolfi
base_url = f'https://{artist_name}.artstation.com' # The base URL of the artist's page for example: https://vy.artstation.com
projects_url = f'{base_url}/projects' # The projects page URL of the artist
driver.get(projects_url)
time.sleep(5)  # Wait for content to render

#### Extract URLs of the images from their thumbnails

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser') # get the HTML content of the page / turn the page into a BeautifulSoup searchable object
project_links = []
for a in soup.find_all('a', href=True): # find all anchor tags 
    href = a['href'] # find the href attribute of the anchor tag / clickable link (thumbnail)
    if re.match(r'^/projects/\w+$', href): # match the href with the regex pattern to find project links (here starts with /projects/ and ends with a combination of letters, digits and underscores)        
        full_link = base_url + href
        if full_link not in project_links:
            project_links.append(full_link)

#### Extract source of images from URLs

In [ ]:
hd_image_urls = []

for i, link in enumerate(project_links):
    print(f"[{i+1}/{len(project_links)}] Visiting: {link}")
    driver.get(link)
    time.sleep(4)

    project_soup = BeautifulSoup(driver.page_source, 'html.parser')
    imgs = project_soup.find_all('img') # find all image tags in the project page
    for img in imgs:
        src = img.get('src') # get the source attribute of the image tag
        if src not in hd_image_urls:
            hd_image_urls.append(src)
            
driver.quit()

# Save the URLs to a text file
with open('urls.txt', 'w') as file:
    for url in hd_image_urls:
        file.write(url + '\n')

print("URLs have been saved to 'urls.txt'.")

#### Save images

In [ ]:
# Folder to save images
save_dir = "downloaded_images"
os.makedirs(save_dir, exist_ok=True)

# Download each image
for i, url in enumerate(hd_image_urls):
    try:
        # Get image content
        response = requests.get(url)
        response.raise_for_status()
        # Create a filename
        ext = url.split('.')[-1].split('?')[0]  # get file extension
        filename = f"image_{i + 1}.{ext}"
        filepath = os.path.join(save_dir, filename)
        # Save the image
        with open(filepath, 'wb') as f:
            f.write(response.content)
        print(f"✅ Saved: {filename}")
    except Exception as e:
        print(f"❌ Failed to download {url}: {e}")